Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
from utils import accuracy, load_pickle

pickle_file = 'notMNIST.pickle'
dic = load_pickle(pickle_file, twoD=True)

('train set', (200000, 28, 28, 1), (200000, 10))
('valid set', (10000, 28, 28, 1), (10000, 10))
('test set', (10000, 28, 28, 1), (10000, 10))


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [33]:
import tensorflow as tf
import numpy as np

image_size = 28
num_labels = 10
num_channels = 1 # grayscale


batch_size = 16 # ？
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():      
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(dic['valid_dataset'])
  tf_test_dataset = tf.constant(dic['test_dataset'])
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print hidden.shape

    #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer2_biases)
    # use max_pool instead
    hidden = tf.nn.max_pool(hidden, [1,3,3,1], [1,2,2,1], padding='SAME')        
    print hidden.shape
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)

    print hidden.shape
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 14, 14, 16)
(16, 7, 7, 16)
(16, 64)
(10000, 14, 14, 16)
(10000, 7, 7, 16)
(10000, 64)
(10000, 14, 14, 16)
(10000, 7, 7, 16)
(10000, 64)


In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (dic['train_labels'].shape[0] - batch_size)
    #print step, batch_size, offset
    batch_data = dic['train_dataset'][offset:(offset + batch_size), :, :, :]
    batch_labels = dic['train_labels'][offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), dic['valid_labels']))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), dic['test_labels']))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [60]:
input = tf.Variable([
    [1.0, 2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0, 8.0],
    [8.0, 7.0, 6.0, 5.0],
    [4.0, 3.0, 2.0, 1.0]])
input=tf.reshape(input,[1,4,4,1])
print input
filter = tf.Variable(tf.random_normal([2,2,1,4]))
print filter
op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    #print(sess.run(input))
    print(sess.run(filter))
    #print(sess.run(op))

Tensor("Reshape_6:0", shape=(1, 4, 4, 1), dtype=float32)
Tensor("Variable_17/read:0", shape=(2, 2, 1, 4), dtype=float32)
[[[[-1.04480505  1.51342654  0.08810008 -1.25273895]]

  [[-0.31495363 -0.92728186  0.97885495  0.32558736]]]


 [[[ 0.59136677 -0.62604582 -0.96583897  0.29905081]]

  [[ 1.01330602  0.95212346 -0.6272487  -0.13555337]]]]


In [56]:
input

<tf.Tensor 'Reshape_4:0' shape=(1, 4, 4, 1) dtype=float32>

In [57]:
op

<tf.Tensor 'Conv2D_4:0' shape=(1, 4, 4, 4) dtype=float32>

In [62]:
filter.shape

AttributeError: 'Variable' object has no attribute 'shape'